In [3]:
import pandas as pd
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output

In [6]:
cotizaciones_df = pd.read_pickle('../etl/Cotizaciones_historicas_SP_500.pkl')

In [7]:
# Inicializar la aplicación Dash
app = JupyterDash(__name__)

# Agrupamos los datos por empresa para su análisis individual
empresas = cotizaciones_df['Company'].unique()

# Layout del Dashboard
app.layout = html.Div([
    html.H1("Dashboard Financiero - Datos de Cotización"),
    html.Label("Seleccione la empresa:"),
    dcc.Dropdown(
        id='empresa-dropdown',
        options=[{'label': empresa, 'value': empresa} for empresa in empresas],
        value=empresas[0]  # Seleccionar la primera empresa como valor predeterminado
    ),
    dcc.Tabs([
        dcc.Tab(label='Precios Históricos', children=[
            dcc.Graph(id='precios-historicos')
        ]),
        dcc.Tab(label='Medias Móviles', children=[
            dcc.Graph(id='medias-moviles')
        ]),
        dcc.Tab(label='RSI', children=[
            dcc.Graph(id='rsi')
        ])
    ])
])

# Callback para actualizar Precios Históricos
@app.callback(
    Output('precios-historicos', 'figure'),
    Input('empresa-dropdown', 'value')
)
def actualizar_precios_historicos(empresa_seleccionada):
    # Filtramos los datos para la empresa seleccionada
    df = cotizaciones_df[cotizaciones_df['Company'] == empresa_seleccionada]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Precio de Cierre'))
    fig.update_layout(title=f"Precios Históricos - {empresa_seleccionada}", xaxis_title="Fecha", yaxis_title="Precio de Cierre")
    return fig

# Callback para actualizar Medias Móviles
@app.callback(
    Output('medias-moviles', 'figure'),
    Input('empresa-dropdown', 'value')
)
def actualizar_medias_moviles(empresa_seleccionada):
    # Filtramos los datos para la empresa seleccionada y calculamos las medias móviles
    df = cotizaciones_df[cotizaciones_df['Company'] == empresa_seleccionada].copy()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['SMA_200'] = df['Close'].rolling(window=200).mean()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Precio de Cierre'))
    fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA_50'], mode='lines', name='SMA 50'))
    fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA_200'], mode='lines', name='SMA 200'))
    fig.update_layout(title=f"Medias Móviles - {empresa_seleccionada}", xaxis_title="Fecha", yaxis_title="Precio")
    return fig

# Callback para actualizar RSI
@app.callback(
    Output('rsi', 'figure'),
    Input('empresa-dropdown', 'value')
)
def actualizar_rsi(empresa_seleccionada):
    # Filtramos los datos para la empresa seleccionada y calculamos el RSI
    df = cotizaciones_df[cotizaciones_df['Company'] == empresa_seleccionada].copy()
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'], y=df['RSI'], mode='lines', name='RSI'))
    fig.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Sobrecompra", annotation_position="top right")
    fig.add_hline(y=30, line_dash="dash", line_color="green", annotation_text="Sobreventa", annotation_position="bottom right")
    fig.update_layout(title=f"RSI (Relative Strength Index) - {empresa_seleccionada}", xaxis_title="Fecha", yaxis_title="RSI")
    return fig

# Ejecutar la aplicación en modo inline para Jupyter Notebook
app.run_server(mode="inline")


/Users/javier/Documents/GitHub/ProyectoPFB/.venv/lib/python3.12/site-packages/dash/dash.py:585: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

